<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=eb0eafca0bd9b3243243ef6f04aaaf192801a75ccfff025d1e36b5d1f30b49d5
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-22 13:21:04
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.41 C
-------------------
Today PnL: 1.10 L (0.78%)
Current PnL: -25.73 L (-16.62%)
CY Booked + Current PnL: -11.44 L (-7.39%)
-------------------
Total profit:  1.66 L
Total loss:  -27.39 L
-------------------
Total Booked + Current PnL: 15.54 L (12.16%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.11%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 90.37 L (63.94%)
Deployed:  1.28 C
Current:  1.41 C
CAGR/XIRR %: 5.55%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,1.27,-18.61,22.98,0.09,18848.0,-18758.0,82019.0,90.39,39.0,M-SC,2.30,253.0,-1.00,0.58,7.76,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,1.05,8.74,10.07,19.69,20996.0,16758.0,208500.0,-1.91,55.0,X-MC,2.47,78.0,0.80,1.48,16.13,XY25,NTT,AC
50,MASFIN,398.61,-1.23,-4.45,27.74,22.05,25969.0,-4365.0,93615.0,-17.80,48.0,H-SC,6.43,164.0,-0.17,0.67,36.08,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,-0.38,-15.89,18.91,0.02,27765.0,-27731.0,146828.0,123.77,43.0,M-SC,12.62,234.0,-1.00,1.04,27.72,OX40N,NTT,PAINTS
43,ITC,452.00,0.52,-1.19,12.12,10.78,28780.0,-2863.0,237455.0,-41.15,50.0,X-LC,2.15,5.0,-0.10,1.69,3.98,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJESHEXPO,518.0,10.83,-56.24,128.70,0.07,77248.0,-77155.0,60022.0,2099.49,77.0,L-SC,3.06,268.0,-1.00,0.43,49.28,OX40N,NTT,JEWELLERY
13,BSOFT,831.7,3.52,-18.65,85.36,50.79,97667.0,-26235.0,114418.0,4.37,72.0,H-SC,7.24,151.0,-0.27,0.81,33.22,XR,ATH,IT
84,WIPRO,420.0,3.20,11.93,53.90,72.27,102229.0,20217.0,189665.0,-3.81,78.0,M-LC,3.38,99.0,0.20,1.35,18.84,XR,NTT,IT
28,GREENPANEL,537.0,3.06,-37.59,129.37,43.14,147807.0,-68827.0,114251.0,155.81,39.0,M-SC,10.08,240.0,-0.47,0.81,7.47,XY24,NTT,MISC
41,INFY,1972.0,2.92,14.82,16.92,34.24,60784.0,46358.0,359246.0,-10.62,75.0,X-LC,5.34,2.0,0.76,2.55,24.33,X40,NTT,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COFFEEDAY,80.00,-3.40,-41.83,132.56,35.27,87553.0,-47501.0,66048.0,-55.73,46.0,L-SC,10.09,270.0,-0.54,0.47,61.65,XR,NTT,HOTELS
44,JCHAC,2282.00,-2.26,-36.53,57.53,-0.01,46669.0,-46683.0,81122.0,16508.70,33.0,M-SC,10.48,233.0,-1.00,0.58,3.83,OX40N,NTT,AC
8,AWL,485.00,-1.76,-23.87,101.62,53.50,233447.0,-72026.0,229725.0,-63.12,34.0,X-MC,4.83,58.0,-0.31,1.63,2.04,XY24,NTT,FMCG
57,RAJOOENG,143.10,-1.51,-37.43,123.14,39.61,78970.0,-38370.0,64130.0,-57.76,33.0,H-SC,30.05,135.0,-0.49,0.46,1.04,AR,ATH,MISC
50,MASFIN,398.61,-1.23,-4.45,27.74,22.05,25969.0,-4365.0,93615.0,-17.80,48.0,H-SC,6.43,164.0,-0.17,0.67,36.08,XR,ATH,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.05,-0.34,108.62,107.91,159825.0,-503.0,147141.0,-18.93,61.0,M-SC,12.03,216.0,-0.0,1.05,9.71,OX40N,ATH,CABLES


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.05,-0.34,108.62,107.91,159825.0,-503.0,147141.0,-18.93,61.0,M-SC,12.03,216.0,-0.00,1.05,9.71,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.38,-15.89,18.91,0.02,27765.0,-27731.0,146828.0,123.77,43.0,M-SC,12.62,234.0,-1.00,1.04,27.72,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,1.27,-18.61,22.98,0.09,18848.0,-18758.0,82019.0,90.39,39.0,M-SC,2.30,253.0,-1.00,0.58,7.76,OX40N,NTT,DURABLES
72,TATAELXSI,9161.00,0.48,-21.61,68.42,32.03,70712.0,-28485.0,103350.0,-15.20,63.0,H-SC,9.34,148.0,-0.40,0.73,15.76,OX40N,NTT,IT
66,SIS,528.00,-0.24,-23.73,58.99,21.26,49956.0,-26346.0,84686.0,1998.45,52.0,H-SC,4.73,166.0,-0.53,0.60,14.42,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,3.2,11.93,53.9,72.27,102229.0,20217.0,189665.0,-3.81,78.0,M-LC,3.38,99.0,0.2,1.35,18.84,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,3.20,11.93,53.90,72.27,102229.0,20217.0,189665.0,-3.81,78.0,M-LC,3.38,99.0,0.20,1.35,18.84,XR,NTT,IT
25,FINCABLES,1641.55,-0.05,-0.34,108.62,107.91,159825.0,-503.0,147141.0,-18.93,61.0,M-SC,12.03,216.0,-0.00,1.05,9.71,OX40N,ATH,CABLES
6,ASIANTILES,137.00,1.90,-2.64,86.60,81.67,79095.0,-2476.0,91334.0,8222.22,73.0,L-SC,11.59,271.0,-0.03,0.65,76.87,XR,NTT,CERAMICS
38,INDIAMART,4810.62,-0.48,-5.21,118.08,106.72,138051.0,-6423.0,116913.0,-53.11,35.0,H-SC,3.98,139.0,-0.05,0.83,16.95,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,0.47,-2.57,39.21,35.63,80060.0,-5392.0,204182.0,-16.30,44.0,H-MC,2.50,119.0,-0.07,1.45,13.67,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,0.59,-17.88,89.32,55.47,178455.0,-43493.0,199793.0,-27.18,32.0,X-MC,14.80,64.0,-0.24,1.42,2.37,X40N,ATH,FINANCE
8,AWL,485.00,-1.76,-23.87,101.62,53.50,233447.0,-72026.0,229725.0,-63.12,34.0,X-MC,4.83,58.0,-0.31,1.63,2.04,XY24,NTT,FMCG
34,HONAUT,58357.33,-0.58,-17.51,73.73,43.32,99064.0,-28512.0,134360.0,-29.17,34.0,X-SC,5.44,90.0,-0.29,0.96,3.16,X40N,ATH,ELECTRICAL
11,BATAINDIA,2096.00,-0.24,-38.82,121.00,35.20,95248.0,-49953.0,78717.0,3.25,35.0,X-SC,16.76,92.0,-0.52,0.56,0.97,X40,NTT,FOOTWEAR
26,GILLETTE,11206.78,0.06,-5.61,40.29,32.42,96557.0,-14241.0,239655.0,-22.30,36.0,X-SC,10.37,85.0,-0.15,1.70,8.64,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,950.0,1.13,0.46,23.91,24.48,60495.0,1155.0,253011.0,-80.87,51.0,X-SC,0.90,86.0,0.02,1.80,23.70,X40N,NTT,MISC
12,BERGEPAINT,680.0,0.97,-3.23,25.18,21.14,55395.0,-7343.0,219996.0,-10.11,45.0,X-MC,1.14,74.0,-0.13,1.56,24.00,XY24,NTT,PAINTS
20,DABUR,735.0,0.26,-3.17,48.41,43.70,116279.0,-7872.0,240196.0,-11.07,38.0,X-MC,2.06,72.0,-0.07,1.71,11.40,XY24,BTT,FMCG
43,ITC,452.0,0.52,-1.19,12.12,10.78,28780.0,-2863.0,237455.0,-41.15,50.0,X-LC,2.15,5.0,-0.10,1.69,3.98,X40,NTT,FMCG
78,UNITDSPR,1644.0,1.09,4.46,15.61,20.77,38180.0,10440.0,244584.0,-4.59,49.0,X-MC,2.16,62.0,0.27,1.74,11.69,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,0.96,-25.68,120.88,64.15,213764.0,-61111.0,176840.0,-56.31,37.0,X-SC,3.61,82.0,-0.29,1.26,0.96,XY24,BTT,CEMENT
11,BATAINDIA,2096.00,-0.24,-38.82,121.00,35.20,95248.0,-49953.0,78717.0,3.25,35.0,X-SC,16.76,92.0,-0.52,0.56,0.97,X40,NTT,FOOTWEAR
19,COLPAL,3726.84,-0.03,-19.86,76.59,41.51,161643.0,-52315.0,211050.0,-4.54,41.0,X-MC,7.67,61.0,-0.32,1.50,1.11,XY25,ATH,FMCG
59,RELAXO,1176.00,0.31,-47.76,191.52,52.31,145247.0,-69321.0,75839.0,-44.34,43.0,X-SC,7.29,91.0,-0.48,0.54,1.51,X40N,NTT,FOOTWEAR
8,AWL,485.00,-1.76,-23.87,101.62,53.50,233447.0,-72026.0,229725.0,-63.12,34.0,X-MC,4.83,58.0,-0.31,1.63,2.04,XY24,NTT,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,1.28,-28.69,100.30,42.84,46497.0,-18648.0,46358.0,-53.17,48.0,X-SC,34.54,83.0,-0.40,0.33,4.88,XY24,NTT,MISC
59,RELAXO,1176.00,0.31,-47.76,191.52,52.31,145247.0,-69321.0,75839.0,-44.34,43.0,X-SC,7.29,91.0,-0.48,0.54,1.51,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.24,-38.82,121.00,35.20,95248.0,-49953.0,78717.0,3.25,35.0,X-SC,16.76,92.0,-0.52,0.56,0.97,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,0.11,-2.23,26.91,24.08,33399.0,-2835.0,124115.0,-10.14,43.0,X-SC,5.85,89.0,-0.08,0.88,16.94,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.58,-17.51,73.73,43.32,99064.0,-28512.0,134360.0,-29.17,34.0,X-SC,5.44,90.0,-0.29,0.96,3.16,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,1.33,-8.55,32.00,20.71,110683.0,-32354.0,345883.0,-21.36,73.0,X-LC,3.65,1.0,-0.29,2.46,15.57,X40,ATH,IT
41,INFY,1972.00,2.92,14.82,16.92,34.24,60784.0,46358.0,359246.0,-10.62,75.0,X-LC,5.34,2.0,0.76,2.55,24.33,X40,NTT,IT
75,TMPV,600.00,1.86,-26.38,67.04,22.98,111494.0,-59588.0,166310.0,-23.08,50.0,X-LC,6.55,3.0,-0.53,1.18,4.60,XY24,NTT,AUTO
81,VBL,671.64,2.63,-2.69,39.42,35.67,121160.0,-8486.0,307356.0,-14.21,58.0,X-LC,3.79,4.0,-0.07,2.19,10.80,X40N,ATH,FMCG
43,ITC,452.00,0.52,-1.19,12.12,10.78,28780.0,-2863.0,237455.0,-41.15,50.0,X-LC,2.15,5.0,-0.10,1.69,3.98,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,1.76,-36.54,109.40,32.89,53603.0,-28210.0,48997.0,-705.35,59.0,L-MC,6.63,259.0,-0.53,0.35,34.93,XR,NTT,BANKS
56,RAJESHEXPO,518.00,10.83,-56.24,128.70,0.07,77248.0,-77155.0,60022.0,2099.49,77.0,L-SC,3.06,268.0,-1.00,0.43,49.28,OX40N,NTT,JEWELLERY
6,ASIANTILES,137.00,1.90,-2.64,86.60,81.67,79095.0,-2476.0,91334.0,8222.22,73.0,L-SC,11.59,271.0,-0.03,0.65,76.87,XR,NTT,CERAMICS
50,MASFIN,398.61,-1.23,-4.45,27.74,22.05,25969.0,-4365.0,93615.0,-17.80,48.0,H-SC,6.43,164.0,-0.17,0.67,36.08,XR,ATH,FINANCE
13,BSOFT,831.70,3.52,-18.65,85.36,50.79,97667.0,-26235.0,114418.0,4.37,72.0,H-SC,7.24,151.0,-0.27,0.81,33.22,XR,ATH,IT


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJESHEXPO,518.00,10.83,-56.24,128.70,0.07,77248.0,-77155.0,60022.0,2099.49,77.0,L-SC,3.06,268.0,-1.00,0.43,49.28,OX40N,NTT,JEWELLERY
13,BSOFT,831.70,3.52,-18.65,85.36,50.79,97667.0,-26235.0,114418.0,4.37,72.0,H-SC,7.24,151.0,-0.27,0.81,33.22,XR,ATH,IT
6,ASIANTILES,137.00,1.90,-2.64,86.60,81.67,79095.0,-2476.0,91334.0,8222.22,73.0,L-SC,11.59,271.0,-0.03,0.65,76.87,XR,NTT,CERAMICS
61,ROUTE,2227.21,-0.03,-45.37,209.46,69.05,152256.0,-60376.0,72690.0,-55.87,56.0,H-SC,21.33,140.0,-0.40,0.52,12.10,SR,ATH,IT
84,WIPRO,420.00,3.20,11.93,53.90,72.27,102229.0,20217.0,189665.0,-3.81,78.0,M-LC,3.38,99.0,0.20,1.35,18.84,XR,NTT,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.47
1,25,44.87
2,50,76.58


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.88
MC    30.78
LC    24.34
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.72
X40      23.10
X40N     13.01
XR        9.86
XY25      9.05
AR        8.90
OX40N     7.59
SR        1.01
MH        0.76
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.28
H-SC    23.65
X-LC    20.87
M-SC    11.65
X-SC     8.03
H-MC     4.80
L-SC     1.55
M-LC     1.35
M-MC     1.35
H-LC     1.11
L-LC     1.01
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.24,-7.61,42.22
IT,13.24,-12.81,69.54
FINANCE,9.46,-19.70,71.76
MISC,7.27,-27.00,78.95
ELECTRICAL,5.98,-11.60,52.47
PAINTS,5.78,-13.36,30.20
INSURANCE,4.75,-1.57,36.33
PHARMA,4.09,-2.28,34.48
AUTO,2.86,-30.37,75.57


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3161747.0,21
AR,1318345.0,10
XR,1275166.0,13
X40,974357.0,14
X40N,858811.0,9
OX40N,724352.0,10
XY25,367572.0,6
SR,278507.0,2
MH,77887.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3650150.0,25
M-SC,1419745.0,15
X-MC,1412187.0,16
X-LC,791524.0,11
X-SC,790271.0,8
H-MC,403651.0,3
L-SC,243896.0,3
M-LC,102229.0,1
H-LC,77076.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1274865.0      6
           AR         906305.0      5
M-SC       XY24       809433.0      6
H-SC       XR         779232.0      7
X-MC       X40        484980.0      7
           XY24       405121.0      3
           X40N       339447.0      4
H-SC       OX40N      333354.0      4
M-SC       OX40N      313750.0      5
X-SC       X40N       304806.0      3
X-LC       X40        297572.0      5
X-SC       XY24       293660.0      3
H-SC       SR         278507.0      2
X-LC       X40N       214558.0      2
H-MC       AR         211856.0      2
X-SC       X40        191805.0      2
H-MC       XY24       191795.0      1
X-LC       XY24       186873.0      2
X-MC       XY25       182639.0      2
M-SC       XR         173454.0      2
L-SC       XR         166648.0      2
M-SC       AR         123108.0      2
M-LC       XR         102229.0      1
X-LC       XY25        92521.0      2
H-SC       MH          77887.0      1
L-SC       OX40N       77248.0      1
H-LC       AR          77076.0      1
M-MC       XY25        55499.0      1
L-MC       XR          53603.0      1
L-LC       XY25        36913.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 39.0 seconds
